In [ ]:
from model import *
from data import *

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.set_default_device(device)

In [ ]:
encoderDim = 128
decoderDim = 256

mixtures = 12

config = Config().load("config.json")